In [1]:
import torch
import os 
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
import mlflow
import mlflow.pytorch

In [2]:
class Config:
    EPOCHS = 10
    BATCH_SIZE = 32
    LR = 0.01
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    GAMMA = 0.7
    SEED = 42
    LOG_INTERVAL = 10
    TEST_BATCH_SIZE = 1000
    DRY_RUN = False

In [3]:
config = Config()

In [4]:
torch.manual_seed(config.SEED)

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
#         output = F.log_softmax(x, dim=1)
        return x



In [5]:
def train_(config, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        pred = model.forward(data)
        loss = F.cross_entropy(pred, target)
        loss.backward()
        optimizer.step()
        if batch_idx % config.LOG_INTERVAL == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100.0 * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")
            if config.DRY_RUN:
                break

In [6]:
def test(model, device, test_loader):
    pass

In [7]:
train_kwargs = {"batch_size": config.BATCH_SIZE}
test_kwargs = {"batch_size": config.TEST_BATCH_SIZE}

In [8]:
if config.DEVICE == "cuda":
    cuda_kwargs = {"num_workers": 1, "pin_memory": True, "shuffle": True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

In [9]:
transform = transforms.Compose(
    [transforms.ToTensor()]
)

In [10]:
train = datasets.MNIST("../data", train=True, download=True, transform=transform)
test = datasets.MNIST("../data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train, **train_kwargs)
test_loader = torch.utils.data.DataLoader(test, **test_kwargs)

In [11]:
model = ConvNet().to(config.DEVICE)
scripted_model = torch.jit.script(model)
optimizer = torch.optim.Adam(model.parameters(), lr=config.LR)

scheduler = StepLR(optimizer, step_size=1, gamma=config.GAMMA)

In [12]:
for i in train_loader:
    print(i[0].shape,i[1].shape)
    break

torch.Size([32, 1, 28, 28]) torch.Size([32])


In [13]:
# training loop

for epoch in range(1, config.EPOCHS + 1):
    train_(config, scripted_model, config.DEVICE, train_loader, optimizer, epoch)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312251
Train Epoch: 1 [320/60000 (1%)]	Loss: 2.027504
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.963831
Train Epoch: 1 [960/60000 (2%)]	Loss: 0.808462
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.877426
Train Epoch: 1 [1600/60000 (3%)]	Loss: 0.852260
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.676226
Train Epoch: 1 [2240/60000 (4%)]	Loss: 0.704212
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.566065
Train Epoch: 1 [2880/60000 (5%)]	Loss: 0.977923
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.946704
Train Epoch: 1 [3520/60000 (6%)]	Loss: 0.377841
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.229552
Train Epoch: 1 [4160/60000 (7%)]	Loss: 0.903893
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.256827
Train Epoch: 1 [4800/60000 (8%)]	Loss: 0.353198
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.611193
Train Epoch: 1 [5440/60000 (9%)]	Loss: 0.622571
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.413605
Train Epoch: 1 [6080/60000 (10%)]	Loss: 0.188413
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.53

In [21]:
with mlflow.start_run() as run:
    mlflow.pytorch.log_model(model, "model")
    model_path = mlflow.get_artifact_uri("model")
    loaded_torch_model = mlflow.pytorch.load_model(model_path)
    model.eval()
    with torch.no_grad():
        test_datapoints, test_target = next(iter(test_loader))
        pred = model(test_datapoints[0].reshape((1,1,28,28)).to(config.DEVICE))
#         print(test_datapoints)
        actual = test_target[0].item()
        predicted = torch.argmax(pred).item()
        print(f"actual: {actual}, predicted: {predicted}")

actual: 4, predicted: 4


In [19]:
print(test_datapoints[0].reshape((1,1,28,28)).shape)


torch.Size([1, 1, 28, 28])
